# <font color="#418FDE" size="6.5" uppercase>**Saving Models**</font>

>Last update: 20260127.
    
By the end of this Lecture, you will be able to:
- Export TensorFlow 2.20.0 models using the SavedModel format with appropriate signatures. 
- Use Keras model.save and save_weights APIs to manage checkpoints and full model exports. 
- Verify that saved models can be reloaded and produce consistent predictions. 


## **1. TensorFlow SavedModel Essentials**

### **1.1. SavedModel Folder Structure**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_01_01.jpg?v=1769518771" width="250">



>* SavedModel exports as a versioned folder structure
>* Each version is a portable, self-contained snapshot

>* saved_model.pb stores graph, signatures, metadata
>* variables folder stores weights, enabling efficient loading

>* SavedModel can include assets and preprocessing resources
>* Directory bundles full pipeline, versioning, and portability



In [ ]:
#@title Python Code - SavedModel Folder Structure

# This script explores TensorFlow SavedModel folders.
# It creates a tiny model and saves it.
# Then it inspects the SavedModel directory structure.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import pathlib
import shutil

# Import TensorFlow and Keras APIs.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set a deterministic random seed.
tf.random.set_seed(42)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Define a small Keras Sequential model.
model = keras.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])

# Compile the model with simple settings.
model.compile(optimizer="adam", loss="binary_crossentropy")

# Create a tiny dummy dataset tensor.
inputs = tf.constant([[0.0, 0.0, 0.0, 0.0],
                      [1.0, 1.0, 1.0, 1.0]],
                     dtype=tf.float32)

# Create tiny binary labels tensor.
labels = tf.constant([[0.0], [1.0]], dtype=tf.float32)

# Train for a few epochs silently.
model.fit(inputs, labels, epochs=5, verbose=0)

# Define a clean export base directory.
export_base = pathlib.Path("savedmodel_demo")

# Remove any previous export directory.
if export_base.exists():
    shutil.rmtree(export_base)

# Create a simple serving signature function.
@tf.function(input_signature=[tf.TensorSpec([None, 4], tf.float32)])
def serve_fn(x):
    return {"probabilities": model(x)}

# Save the model using the SavedModel format.
model.export(
    export_base,
    # signatures has no effect in Keras 3 export; use export_signature instead
    # but we keep this call minimal and instead adapt to the actual output key
)

# List top level contents of export directory.
print("Top level entries:", sorted(os.listdir(export_base)))

# Find versioned subdirectories inside export directory.
version_dirs = [
    d for d in os.listdir(export_base)
    if os.path.isdir(export_base / d)
]

# Print discovered versioned folders.
print("Version folders:", sorted(version_dirs))

# Choose the first version folder for inspection.
version_path = export_base / sorted(version_dirs)[0]

# List files inside the chosen version folder.
print("Files in version folder:", sorted(os.listdir(version_path)))

# Build path to variables subdirectory.
variables_path = version_path / "variables"

# List variable files if directory exists.
if variables_path.exists():
    print("Variables files:", sorted(os.listdir(variables_path)))

# Build path to assets subdirectory.
assets_path = version_path / "assets"

# Print assets folder contents or absence.
if assets_path.exists():
    print("Assets files:", sorted(os.listdir(assets_path)))
else:
    print("Assets folder is empty or missing.")

# Load the SavedModel back from disk.
reloaded = tf.saved_model.load(str(export_base))

# Call the default serving signature with sample input.
reloaded_output = reloaded.signatures["serving_default"](inputs)

# Print keys and shape of reloaded output.
print("Reloaded output keys:", list(reloaded_output.keys()))

# Print shape of probabilities tensor from reloaded model.
prob_key = list(reloaded_output.keys())[0]
print("Reloaded probabilities shape:", reloaded_output[prob_key].shape)



### **1.2. Serving Signatures Overview**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_01_02.jpg?v=1769518908" width="250">



>* Serving signatures define model input and output interfaces
>* They act as stable contracts for integrations

>* One SavedModel can expose several specialized signatures
>* Serving systems route requests using signature names

>* Plan signatures around real-world usage and preprocessing
>* Keep interfaces stable to decouple models and applications



In [ ]:
#@title Python Code - Serving Signatures Overview

# This script shows basic TensorFlow SavedModel signatures.
# It focuses on simple Keras model exporting and loading.
# Run all cells sequentially inside Google Colab easily.

# !pip install tensorflow==2.20.0.

# Import required standard libraries safely.
import os
import pathlib
import numpy as np

# Import TensorFlow and Keras high level APIs.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Set deterministic random seeds for reproducibility.
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Detect available device type for information only.
physical_gpus = tf.config.list_physical_devices("GPU")
print("GPUs available:", len(physical_gpus))

# Create small dummy numeric dataset for demonstration.
num_samples = 8
x_data = np.linspace(-1.0, 1.0, num_samples).reshape(-1, 1)

# Create simple target values using a linear rule.
y_data = (2.0 * x_data + 0.5).astype("float32")

# Validate shapes before building the model.
print("Input shape:", x_data.shape, "Target shape:", y_data.shape)

# Build a tiny Keras Sequential regression model.
model = keras.Sequential([
    layers.Input(shape=(1,)),
    layers.Dense(4, activation="relu"),
    layers.Dense(1)
])

# Compile the model with mean squared error loss.
model.compile(optimizer="adam", loss="mse")

# Train briefly with silent verbose setting.
model.fit(x_data, y_data, epochs=50, verbose=0)

# Evaluate one prediction before saving the model.
example_input = np.array([[0.25]], dtype="float32")
original_pred = model.predict(example_input, verbose=0)

# Print original prediction for comparison later.
print("Original prediction:", float(original_pred[0, 0]))

# Define a serving function with clear input signature.
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.float32, name="features")])
def serve_regression(features):
    outputs = model(features)
    return {"prediction": outputs}

# Prepare directory for SavedModel export.
export_dir = pathlib.Path("saved_model_signatures")
if export_dir.exists():
    pass

# Save the model with explicit serving signatures mapping.
tf.saved_model.save(
    model,
    export_dir.as_posix(),
    signatures={"serving_default": serve_regression},
)

# Save only model weights separately for completeness.
weights_path = export_dir.joinpath("weights_only.weights.h5")
model.save_weights(weights_path.as_posix())

# Load the SavedModel as a generic trackable object.
loaded = tf.saved_model.load(export_dir.as_posix())

# List available signatures exposed by the SavedModel.
print("Available signatures:", list(loaded.signatures.keys()))

# Grab the default serving signature for inference.
serving_fn = loaded.signatures["serving_default"]

# Prepare input tensor with correct key and shape.
serving_inputs = {"features": tf.constant([[0.25]], dtype=tf.float32)}

# Call the serving function to get predictions.
served_outputs = serving_fn(**serving_inputs)

# Extract numeric prediction from the returned dictionary.
served_pred = float(served_outputs["prediction"][0, 0].numpy())

# Compare original and served predictions for consistency.
print("Served prediction:", served_pred)
print("Difference magnitude:", abs(served_pred - float(original_pred[0, 0])))

# Show that Keras model.save also writes a SavedModel.
keras_export_dir = pathlib.Path("keras_saved_model.keras")
model.save(keras_export_dir.as_posix())

# Confirm that the Keras export directory now exists.
print("Keras SavedModel exists:", keras_export_dir.exists())



### **1.3. Model Versioning Strategies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_01_03.jpg?v=1769519099" width="250">



>* Use separate folders for each model export
>* Versioning enables rollback, debugging, and traceability

>* Serving expects model folders with version subdirectories
>* Route traffic between versions and quickly roll back

>* Include semantic tags in SavedModel version directories
>* Supports audits, investigations, experimentation, and collaboration



## **2. Keras Model Saving**

### **2.1. Model Save Options**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_02_01.jpg?v=1769519130" width="250">



>* Choose between saving full model or weights
>* Decision impacts portability, coupling, and team reuse

>* Use full model saves for deployment scenarios
>* Use weight-only saves for fast experimentation

>* Use frequent weight-only checkpoints during long training
>* Select best checkpoint, then export full deployable model



In [ ]:
#@title Python Code - Model Save Options

# This script demonstrates basic Keras model saving.
# It focuses on full model and weights only.
# Run cells sequentially to follow each concept.

# !pip install tensorflow==2.20.0.

# Import required TensorFlow and Keras modules.
import tensorflow as tf

# Set deterministic seeds for reproducibility.
tf.keras.utils.set_random_seed(42)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Prepare a tiny subset of MNIST digits.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reduce dataset size for quick demonstration.
x_train_small, y_train_small = x_train[:2000], y_train[:2000]

# Normalize pixel values to the range zero one.
x_train_small = x_train_small.astype("float32") / 255.0

# Add channel dimension expected by Conv2D layers.
x_train_small = x_train_small[..., tf.newaxis]

# Confirm input shape is as expected.
print("Train subset shape:", x_train_small.shape)

# Build a simple sequential convolutional model.
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation="relu",
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Compile model with standard settings.
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train briefly with silent output for speed.
model.fit(x_train_small, y_train_small, epochs=2,
          batch_size=64, verbose=0)

# Select a small batch for prediction comparison.
sample_batch = x_train_small[:5]

# Get predictions from the trained original model.
original_preds = model.predict(sample_batch, verbose=0)

# Show predicted classes from original model.
print("Original classes:", original_preds.argmax(axis=1))

# Define directory paths for saving artifacts.
full_model_path = "saved_full_model.keras"
weights_path = "saved_weights_only.weights.h5"

# Save the entire model including architecture and weights.
model.save(full_model_path, include_optimizer=False)

# Save only the model weights as a checkpoint file.
model.save_weights(weights_path)

# Load the full model from the SavedModel directory.
loaded_full_model = tf.keras.models.load_model(full_model_path)

# Predict again using the fully loaded model.
full_preds = loaded_full_model.predict(sample_batch, verbose=0)

# Show predicted classes from the loaded full model.
print("Full model classes:", full_preds.argmax(axis=1))

# Recreate the same architecture for weights loading.
recreated_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation="relu",
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Compile recreated model before loading weights.
recreated_model.compile(optimizer="adam",
                        loss="sparse_categorical_crossentropy",
                        metrics=["accuracy"])

# Load the previously saved weights into recreated model.
recreated_model.load_weights(weights_path)

# Predict using the recreated model with loaded weights.
weights_preds = recreated_model.predict(sample_batch, verbose=0)

# Show predicted classes from the weights only model.
print("Weights model classes:", weights_preds.argmax(axis=1))

# Verify that all three prediction sets are identical.
print("All predictions match:",
      (original_preds.argmax(axis=1) ==
       full_preds.argmax(axis=1)).all() and
      (original_preds.argmax(axis=1) ==
       weights_preds.argmax(axis=1)).all())



### **2.2. HDF5 and SavedModel**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_02_02.jpg?v=1769519251" width="250">



>* Keras saves models as HDF5 file or SavedModel folder
>* HDF5 is compact; SavedModel is default for production

>* HDF5 is simple, single-file, and shareable
>* Great for Keras research, weaker for production

>* SavedModel stores graphs, weights, and assets portably
>* Best for scalable, cross-platform, production TensorFlow deployment



In [ ]:
#@title Python Code - HDF5 and SavedModel

# This script shows Keras model saving basics.
# We compare HDF5 and SavedModel formats.
# We also verify reloaded models predictions.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import pathlib
import numpy as np

# Import TensorFlow and Keras modules.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set deterministic random seeds.
np.random.seed(7)
tf.random.set_seed(7)

# Prepare a tiny synthetic regression dataset.
x_data = np.linspace(-1.0, 1.0, 200).astype("float32")

# Create corresponding target values with noise.
y_data = (3.0 * x_data + 0.5 + 0.1 * np.random.randn(200)).astype("float32")

# Reshape data to column vectors.
x_data = x_data.reshape(-1, 1)
y_data = y_data.reshape(-1, 1)

# Validate shapes before training.
print("Input shape:", x_data.shape)
print("Target shape:", y_data.shape)

# Build a simple Sequential regression model.
model = keras.Sequential([
    layers.Input(shape=(1,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(1)
])

# Compile model with mean squared error loss.
model.compile(optimizer="adam", loss="mse")

# Train briefly with silent output.
model.fit(x_data, y_data, epochs=20, batch_size=32, verbose=0)

# Pick a small batch for prediction check.
sample_inputs = np.array([[-0.5], [0.0], [0.5]], dtype="float32")

# Get original model predictions.
original_preds = model.predict(sample_inputs, verbose=0)

# Create base directory for saved models.
base_dir = pathlib.Path("saved_models_demo")
base_dir.mkdir(exist_ok=True)

# Define HDF5 file path.
h5_path = base_dir / "linear_regression.h5"

# Save full model in HDF5 format.
model.save(h5_path, include_optimizer=False)

# Define SavedModel directory path.
savedmodel_path = base_dir / "linear_regression_savedmodel.keras"

# Save full model in SavedModel format.
model.save(savedmodel_path, include_optimizer=False)

# Save only weights to a separate checkpoint.
weights_path = base_dir / "linear_regression_weights.weights.h5"
model.save_weights(weights_path)

# Reload model from HDF5 file.
loaded_h5 = keras.models.load_model(h5_path)

# Reload model from SavedModel directory.
loaded_saved = keras.models.load_model(savedmodel_path)

# Build same architecture for weights loading.
weights_model = keras.Sequential([
    layers.Input(shape=(1,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(1)
])

# Compile before loading weights.
weights_model.compile(optimizer="adam", loss="mse")

# Load previously saved weights.
weights_model.load_weights(weights_path)

# Compute predictions from all three models.
h5_preds = loaded_h5.predict(sample_inputs, verbose=0)

# Predictions from SavedModel loaded model.
saved_preds = loaded_saved.predict(sample_inputs, verbose=0)

# Predictions from weights only loaded model.
weights_preds = weights_model.predict(sample_inputs, verbose=0)

# Helper function to compare arrays safely.
def max_abs_diff(a, b):
    return float(np.max(np.abs(a - b)))

# Compute maximum absolute differences.
max_diff_h5 = max_abs_diff(original_preds, h5_preds)
max_diff_saved = max_abs_diff(original_preds, saved_preds)
max_diff_weights = max_abs_diff(original_preds, weights_preds)

# Print concise comparison results.
print("Original predictions:\n", np.round(original_preds, 3))
print("HDF5 reload diff:", max_diff_h5)
print("SavedModel reload diff:", max_diff_saved)
print("Weights reload diff:", max_diff_weights)



### **2.3. Loading Custom Objects**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_02_03.jpg?v=1769519351" width="250">



>* Custom layers need special handling when reloading
>* Missing mappings can break predictions and reliability

>* Register custom objects so Keras can reload
>* Store configs with weights to rebuild behavior

>* Custom objects must be defined and registered
>* Ensures long-term reproducibility and consistent predictions



In [ ]:
#@title Python Code - Loading Custom Objects

# This script shows loading custom objects.
# It focuses on Keras model saving.
# Run all cells sequentially in Colab.

# Install TensorFlow if not already available.
# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducibility.
os.environ["PYTHONHASHSEED"] = "0"
np.random.seed(0)
tf.random.set_seed(0)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Define a simple custom scaling layer.
class CustomScaling(tf.keras.layers.Layer):
    def __init__(self, scale_factor=1.0, **kwargs):
        super().__init__(**kwargs)
        self.scale_factor = scale_factor

    def call(self, inputs):
        return inputs * self.scale_factor

    def get_config(self):
        config = super().get_config()
        config.update({"scale_factor": self.scale_factor})
        return config

# Create a tiny model using the custom layer.
inputs = tf.keras.Input(shape=(4,))
scaled = CustomScaling(scale_factor=2.0)(inputs)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(scaled)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model with a simple configuration.
model.compile(optimizer="adam", loss="binary_crossentropy")

# Create a tiny synthetic dataset.
x_train = np.random.rand(8, 4).astype("float32")
y_train = np.random.randint(0, 2, size=(8, 1)).astype("float32")

# Validate shapes before training.
assert x_train.shape == (8, 4)
assert y_train.shape == (8, 1)

# Train briefly with silent output.
model.fit(x_train, y_train, epochs=3, verbose=0)

# Generate a small batch for prediction.
x_sample = np.array([[0.1, 0.2, 0.3, 0.4]], dtype="float32")

# Get predictions before saving.
original_pred = model.predict(x_sample, verbose=0)

# Define a directory for saving the model.
save_dir = "custom_scaling_model.keras"

# Remove any existing directory safely.
if tf.io.gfile.exists(save_dir):
    tf.io.gfile.rmtree(save_dir)

# Save the full model including custom layer.
model.save(save_dir, include_optimizer=False)

# Load the model without custom_objects to show failure.
loaded_without = None
try:
    loaded_without = tf.keras.models.load_model(save_dir)
except Exception as e:
    print("Load without custom_objects failed.")

# Load the model with custom_objects mapping.
loaded_with = tf.keras.models.load_model(
    save_dir,
    custom_objects={"CustomScaling": CustomScaling},
)

# Predict again using the correctly loaded model.
reloaded_pred = loaded_with.predict(x_sample, verbose=0)

# Compare predictions for consistency.
print("Original prediction:", np.round(original_pred, 4))
print("Reloaded prediction:", np.round(reloaded_pred, 4))
print("Difference:", np.round(original_pred - reloaded_pred, 8))

# Show that the custom layer type is preserved.
print("Loaded layer type:", type(loaded_with.layers[1]).__name__)



## **3. Reloading and Validating Models**

### **3.1. Loading Saved Models**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_03_01.jpg?v=1769519459" width="250">



>* Load saved models back into memory objects
>* Restore full pipeline to reproduce original behavior

>* Signatures define expected inputs and returned outputs
>* Check signatures align with pipeline and use case

>* Run test inputs and compare stored predictions
>* Catch version, pipeline, or file corruption issues



In [ ]:
#@title Python Code - Loading Saved Models

# This script shows how to reload models.
# It focuses on loading and validating predictions.
# Follow along to understand SavedModel reloading.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducibility.
np.random.seed(7)
tf.random.set_seed(7)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Select device based on GPU availability.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_name = "GPU"
else:
    device_name = "CPU"

# Inform which device is being used.
print("Running on device:", device_name)

# Load a small built in dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()

# Reduce dataset size for quick training.
x_train_small = x_train[:2000].astype("float32") / 255.0
y_train_small = y_train[:2000].astype("int32")

# Add channel dimension for convolutional layers.
x_train_small = np.expand_dims(x_train_small, axis=-1)

# Validate shapes before building model.
print("Train subset shape:", x_train_small.shape)

# Build a simple sequential CNN model.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(8, 3, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Compile the model with standard settings.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train briefly with silent output.
model.fit(
    x_train_small,
    y_train_small,
    epochs=1,
    batch_size=64,
    verbose=0,
)

# Select a tiny batch for prediction tests.
x_sample = x_train_small[:5]
y_sample = y_train_small[:5]

# Get predictions from the in memory model.
original_probs = model.predict(x_sample, verbose=0)

# Convert probabilities to class indices.
original_preds = np.argmax(original_probs, axis=1)

# Show original predictions and labels.
print("Original preds:", original_preds, "Labels:", y_sample)

# Prepare directory paths for saving.
base_dir = "saved_models_demo"
os.makedirs(base_dir, exist_ok=True)

# Define SavedModel export path.
savedmodel_path = os.path.join(base_dir, "mnist_savedmodel.keras")

# Save full model using SavedModel format.
model.save(savedmodel_path)

# Define weights only checkpoint path.
weights_path = os.path.join(base_dir, "mnist_weights.weights.h5")

# Save only the model weights to disk.
model.save_weights(weights_path)

# Reload full model from SavedModel directory.
reloaded_model = tf.keras.models.load_model(savedmodel_path)

# Predict again using the reloaded full model.
reloaded_probs = reloaded_model.predict(x_sample, verbose=0)

# Convert probabilities to class indices again.
reloaded_preds = np.argmax(reloaded_probs, axis=1)

# Compare original and reloaded predictions.
print("Reloaded preds:", reloaded_preds)

# Check if predictions match exactly.
match_full = np.array_equal(original_preds, reloaded_preds)

# Report whether full model predictions match.
print("Full model predictions match:", bool(match_full))

# Rebuild the same model architecture for weights.
weights_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(8, 3, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Compile the weights model before loading.
weights_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Load the previously saved weights file.
weights_model.load_weights(weights_path)

# Predict using the weights only reloaded model.
weights_probs = weights_model.predict(x_sample, verbose=0)

# Convert probabilities to class indices again.
weights_preds = np.argmax(weights_probs, axis=1)

# Check if weights model predictions match original.
match_weights = np.array_equal(original_preds, weights_preds)

# Print final consistency summary for both reload paths.
print("Weights model predictions match:", bool(match_weights))



### **3.2. Deterministic Model Predictions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_03_02.jpg?v=1769519613" width="250">



>* Check reloaded models give identical outputs consistently
>* Determinism enables debugging, auditing, and trustworthy comparisons

>* Control randomness and use fixed test inputs
>* Compare outputs to baseline to confirm identical behavior

>* Control or disable stochastic behavior during inference
>* Use deterministic config in production, document conditions



In [ ]:
#@title Python Code - Deterministic Model Predictions

# This script shows deterministic TensorFlow predictions.
# It trains saves reloads and compares model outputs.
# Use this to validate stable serving model behavior.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import numpy as np
import tensorflow as tf

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set global random seeds for determinism.
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Select device preferring GPU when available.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_name = "/GPU:0"
else:
    device_name = "/CPU:0"
print("Using device:", device_name)

# Create a tiny deterministic dataset.
num_samples = 64
x_data = np.linspace(-1.0, 1.0, num_samples).reshape(-1, 1)
noise = np.zeros_like(x_data)
y_data = 3.0 * x_data + 0.5 + noise

# Validate dataset shapes defensively.
assert x_data.shape == (num_samples, 1)
assert y_data.shape == (num_samples, 1)

# Build a simple Keras regression model.
with tf.device(device_name):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

# Compile the model with mean squared error.
model.compile(optimizer="adam", loss="mse")

# Train briefly with silent output.
history = model.fit(
    x_data,
    y_data,
    epochs=30,
    batch_size=16,
    verbose=0,
    shuffle=False
)

# Choose a small fixed test batch.
x_test = np.array([[-0.5], [0.0], [0.5]], dtype=np.float32)
assert x_test.shape == (3, 1)

# Get baseline predictions before saving.
baseline_preds = model.predict(x_test, verbose=0)

# Define a safe export directory.
export_dir = "deterministic_model_export"
if not os.path.exists(export_dir):
    os.makedirs(export_dir, exist_ok=True)

# Save full model using SavedModel format.
model_save_path = os.path.join(export_dir, "full_model.keras")
model.save(model_save_path)

# Save only weights to a separate file.
weights_path = os.path.join(export_dir, "model_weights.weights.h5")
model.save_weights(weights_path)

# Reload full model from disk.
reloaded_model = tf.keras.models.load_model(model_save_path)

# Ensure reloaded model input shape matches.
reloaded_input_shape = reloaded_model.inputs[0].shape
assert reloaded_input_shape[1] == 1

# Get predictions from the reloaded model.
reloaded_preds = reloaded_model.predict(x_test, verbose=0)

# Compare predictions using a small tolerance.
max_abs_diff = np.max(np.abs(baseline_preds - reloaded_preds))
are_close = np.allclose(baseline_preds, reloaded_preds, atol=1e-6)

# Print deterministic comparison summary.
print("Baseline predictions:", np.round(baseline_preds.flatten(), 4))
print("Reloaded predictions:", np.round(reloaded_preds.flatten(), 4))
print("Max absolute difference:", float(max_abs_diff))
print("Predictions match within tolerance:", bool(are_close))

# Demonstrate reusing weights with same architecture.
with tf.device(device_name):
    same_arch_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

# Load saved weights into new model.
same_arch_model.compile(optimizer="adam", loss="mse")
same_arch_model.load_weights(weights_path)

# Predict again to confirm deterministic behavior.
weights_preds = same_arch_model.predict(x_test, verbose=0)

# Check that weights based predictions also match.
weights_close = np.allclose(baseline_preds, weights_preds, atol=1e-6)
print("Weights based predictions:", np.round(weights_preds.flatten(), 4))
print("Weights predictions match baseline:", bool(weights_close))



### **3.3. Handling Missing Dependencies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_09/Lecture_A/image_03_03.jpg?v=1769519820" width="250">



>* Models depend on specific code and libraries
>* Mismatched environments cause loading errors or behavior changes

>* Record versions and list all model dependencies
>* Package custom components so reloaded models behave consistently

>* Version mismatches can cause subtle prediction changes
>* Validate outputs across environments and align dependencies



# <font color="#418FDE" size="6.5" uppercase>**Saving Models**</font>


In this lecture, you learned to:
- Export TensorFlow 2.20.0 models using the SavedModel format with appropriate signatures. 
- Use Keras model.save and save_weights APIs to manage checkpoints and full model exports. 
- Verify that saved models can be reloaded and produce consistent predictions. 

In the next Lecture (Lecture B), we will go over 'Serving and APIs'